In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score

# =====================================================
# 데이터 로드 + 시간 파생
# =====================================================
df = pd.read_csv("new_flight_weather_merged.csv")

df["departure_datetime"] = pd.to_datetime(df["departure_datetime"])

# ⏱ 시간 파생
df["dep_hour"] = df["departure_datetime"].dt.hour
df["dep_minute"] = df["departure_datetime"].dt.minute   # ✅ 분 추가
df["dep_weekday"] = df["departure_datetime"].dt.weekday
df["is_weekend"] = df["dep_weekday"].isin([5, 6]).astype(int)

# =====================================================
# 컬럼 정의
# =====================================================
num_cols = [
    "기온(°C)",
    "풍속_ms",
    "dep_hour",
    "dep_minute",      # ✅ 분 포함
    "dep_weekday",
    "is_weekend"
]
num_cols = [c for c in num_cols if c in df.columns]

cat_cols = ["항공사", "출발지", "도착지", "flight_type"]
cat_cols = [c for c in cat_cols if c in df.columns]

# 👉 LightGBM용 category 변환
for c in cat_cols:
    df[c] = df[c].astype("category")

X_cols = num_cols + cat_cols

# =====================================================
# Train / Test 분리 (시간 기준)
# =====================================================
df = df.sort_values("departure_datetime")
split_date = df["departure_datetime"].quantile(0.8)

train_df = df[df["departure_datetime"] <= split_date]
test_df  = df[df["departure_datetime"] > split_date]

X_train = train_df[X_cols]
y_train = train_df["is_delay"]

X_test  = test_df[X_cols]
y_test  = test_df["is_delay"]

# =====================================================
# LightGBM 모델
# =====================================================
lgbm = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,

    class_weight="balanced",
    objective="binary",
    metric="aucpr",
    random_state=42,
    n_jobs=-1
)

# ✅ categorical_feature 지정
lgbm.fit(
    X_train,
    y_train,
    categorical_feature=cat_cols
)

print("✅ LightGBM 학습 완료 (minute 포함)")

# =====================================================
# 평가
# =====================================================
y_prob = lgbm.predict_proba(X_test)[:, 1]

for t in [0.3, 0.35, 0.4, 0.45]:
    print(f"\n===== Threshold = {t} =====")
    y_pred = (y_prob >= t).astype(int)
    print(classification_report(y_test, y_pred))

print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("PR-AUC :", average_precision_score(y_test, y_prob))

# =====================================================
# 💾 모델 저장
# =====================================================
import joblib

joblib.dump(lgbm, "final_delay_classifier_with_minute.joblib")
print("✅ 분 포함 분류 모델 저장 완료: final_delay_classifier_with_minute.joblib")


C:\Users\Admin\AppData\Local\Temp\ipykernel_10276\3272522046.py:9: DtypeWarning: Columns (28,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new_flight_weather_merged.csv")


[LightGBM] [Info] Number of positive: 336343, number of negative: 1938804
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020081 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 826
[LightGBM] [Info] Number of data points in the train set: 2275147, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
✅ LightGBM 학습 완료 (minute 포함)

===== Threshold = 0.3 =====
              precision    recall  f1-score   support

           0       0.93      0.38      0.54    422243
           1       0.34      0.92      0.50    146544

    accuracy                           0.52    568787
   macro avg       0.64      0.65      0.52    568787
weighted avg       0.78      0.52      0.53    568787


===== Threshold = 0.35 =====
              precision

In [3]:
# =====================================================
# 💾 최종 분류 모델 저장 (LightGBM Classifier)
# =====================================================
# ✔ 학습된 LightGBM 분류 모델 저장
# ✔ 이후 바로 load 해서 predict_proba 가능
# =====================================================

import joblib

joblib.dump(lgbm, "final_delay_classifier.joblib")

print("✅ 분류 모델 저장 완료: final_delay_classifier.joblib")


✅ 분류 모델 저장 완료: final_delay_classifier.joblib
